In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
#from sklearn.externals import joblib

In [45]:
train = pd.read_csv(r'C:\Users\Eugenio\Desktop\Kaggle\train_new.csv')

C:\Users\Eugenio\anaconda3\envs\env_eug\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,8,10,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [47]:
test = pd.read_csv(r'C:\Users\Eugenio\Desktop\Kaggle\test_new.csv')

# train / test

In [48]:
train.shape

(2022540, 48)

In [49]:
test.shape

(150000, 48)

In [6]:
train

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-12-28,81461,N,ES,V,54,1998-01-05,0.0,215,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-10-28,346308,N,ES,V,59,2015-01-02,0.0,19,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2016-02-28,270540,N,ES,V,43,2001-09-04,0.0,84,1.0,...,0,0,0,0,0,0,0,0.0,0.0,1
3,2015-09-28,1335902,N,ES,V,21,2014-10-17,0.0,11,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-08-28,258215,N,ES,H,42,2001-07-02,0.0,170,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269586,2015-06-28,734657,N,ES,V,37,2007-11-27,0.0,92,1.0,...,0,0,0,0,0,0,1,0.0,0.0,0
269587,2015-02-28,953472,N,ES,H,28,2011-10-03,0.0,45,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
269588,2015-03-28,1358822,N,ES,V,42,2014-11-21,0.0,8,1.0,...,0,0,0,1,0,0,0,0.0,0.0,1
269589,2016-02-28,118862,N,ES,H,58,1999-01-15,0.0,205,1.0,...,0,0,0,0,0,1,0,0.0,0.0,1


In [7]:
train.dtypes

fecha_dato                object
ncodpers                   int64
ind_empleado              object
pais_residencia           object
sexo                      object
age                       object
fecha_alta                object
ind_nuevo                float64
antiguedad                object
indrel                   float64
ult_fec_cli_1t            object
indrel_1mes               object
tiprel_1mes               object
indresi                   object
indext                    object
conyuemp                  object
canal_entrada             object
indfall                   object
tipodom                  float64
cod_prov                 float64
nomprov                   object
ind_actividad_cliente    float64
renta                    float64
segmento                  object
ind_ahor_fin_ult1          int64
ind_aval_fin_ult1          int64
ind_cco_fin_ult1           int64
ind_cder_fin_ult1          int64
ind_cno_fin_ult1           int64
ind_ctju_fin_ult1          int64
ind_ctma_f

In [10]:
train.groupby(['ncodpers','fecha_dato'])['ncodpers'].count()

ncodpers  fecha_dato
15892     2015-01-28    1
          2015-12-28    1
          2016-03-28    1
15900     2015-01-28    1
          2015-02-28    1
                       ..
1548033   2016-04-28    1
1548092   2016-04-28    1
1548094   2016-04-28    1
1548100   2016-04-28    1
1548145   2016-04-28    1
Name: ncodpers, Length: 269591, dtype: int64

In [11]:
train.fecha_dato.describe()

count         269591
unique            16
top       2016-04-28
freq           19937
Name: fecha_dato, dtype: object

In [50]:
def transform_dataset_train(df,inputString, numberMonths):
    df = df.sort_values(['ncodpers','fecha_dato'])
    df['rowNB']  = df.groupby(['ncodpers'])['fecha_dato'].cumcount()+1
    df_aux = df.copy()
    df_aux.columns = [str(col) + f"{inputString}"  for col in df_aux.columns]
    df2 = df.merge(df_aux, left_on=['ncodpers'], right_on=['ncodpers' + f"{inputString}"],how='left')    
    df2['rowDif'] = df2['rowNB' + f"{inputString}"]- df2.rowNB
    df3 = df2[(df2.ncodpers == (df2['ncodpers' + f"{inputString}"])) & (df2.rowDif==numberMonths)]
    return df3

In [51]:
df_train2 = transform_dataset_train(train,'_1_before',1)

MemoryError: Unable to allocate 4.60 GiB for an array with shape (21, 29402386) and data type int64

In [33]:
df_train2

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB,fecha_dato_1_before,ncodpers_1_before,ind_empleado_1_before,pais_residencia_1_before,sexo_1_before,age_1_before,fecha_alta_1_before,ind_nuevo_1_before,antiguedad_1_before,indrel_1_before,ult_fec_cli_1t_1_before,indrel_1mes_1_before,tiprel_1mes_1_before,indresi_1_before,indext_1_before,conyuemp_1_before,canal_entrada_1_before,indfall_1_before,tipodom_1_before,cod_prov_1_before,nomprov_1_before,ind_actividad_cliente_1_before,renta_1_before,segmento_1_before,ind_ahor_fin_ult1_1_before,ind_aval_fin_ult1_1_before,ind_cco_fin_ult1_1_before,ind_cder_fin_ult1_1_before,ind_cno_fin_ult1_1_before,ind_ctju_fin_ult1_1_before,ind_ctma_fin_ult1_1_before,ind_ctop_fin_ult1_1_before,ind_ctpp_fin_ult1_1_before,ind_deco_fin_ult1_1_before,ind_deme_fin_ult1_1_before,ind_dela_fin_ult1_1_before,ind_ecue_fin_ult1_1_before,ind_fond_fin_ult1_1_before,ind_hip_fin_ult1_1_before,ind_plan_fin_ult1_1_before,ind_pres_fin_ult1_1_before,ind_reca_fin_ult1_1_before,ind_tjcr_fin_ult1_1_before,ind_valo_fin_ult1_1_before,ind_viv_fin_ult1_1_before,ind_nomina_ult1_1_before,ind_nom_pens_ult1_1_before,ind_recibo_ult1_1_before,rowNB_1_before,rowDif
1,2015-01-28,15892,F,ES,H,61,1995-01-16,0.0,246,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,430477.41,01 - TOP,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,1,2015-12-28,15892,F,ES,H,61,1995-01-16,0.0,251,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,430477.41,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,2,1
5,2015-12-28,15892,F,ES,H,61,1995-01-16,0.0,251,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,430477.41,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,2,2016-03-28,15892,F,ES,H,62,1995-01-16,0.0,254,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,430477.41,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,3,1
10,2015-01-28,15900,B,ES,V,48,1995-01-16,0.0,246,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,105327.03,02 - PARTICULARES,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,1,1,2015-02-28,15900,B,ES,V,48,1995-01-16,0.0,246,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,105327.03,02 - PARTICULARES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,1,2,1
15,2015-02-28,15900,B,ES,V,48,1995-01-16,0.0,246,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,105327.03,02 - PARTICULARES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,1,2,2015-11-28,15900,B,ES,V,48,1995-01-16,0.0,250,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,105327.03,02 - PARTICULARES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,1,3,1
20,2015-11-28,15900,B,ES,V,48,1995-01-16,0.0,250,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,105327.03,02 - PARTICULARES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,1,3,2016-01-28,15900,B,ES,V,48,1995-01-16,0.0,252,1.0,NaN,1.0,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,105327.03,02 - PARTICULARES,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804972,2016-03-28,1542163,N,ES,H,32,2016-03-30,1.0,0,1.0,NaN,NaN,NaN,S,N,NaN

In [25]:
df_train2.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1', 'rowNB',
       'fecha_dato_1_before', 'ncodpers_1_before', 'ind_empleado_1_before',
       'pais

In [13]:
df_test2 = transform_dataset_train(test,'_next',1)

In [14]:
df_test2

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_pres_fin_ult1_next,ind_reca_fin_ult1_next,ind_tjcr_fin_ult1_next,ind_valo_fin_ult1_next,ind_viv_fin_ult1_next,ind_nomina_ult1_next,ind_nom_pens_ult1_next,ind_recibo_ult1_next,rowNB_next,rowDif


In [15]:
test

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB
0,2016-05-28,1492740,N,ES,H,20,2015-10-28,0.0,7,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
1,2016-05-28,968856,N,ES,V,24,2011-10-25,0.0,55,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
2,2016-05-28,76198,N,ES,V,65,1997-11-12,0.0,222,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
3,2016-05-28,132411,N,ES,V,42,1999-05-24,0.0,204,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
4,2016-05-28,853734,N,ES,V,42,2009-10-07,0.0,72,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136179,2016-05-28,391341,N,ES,H,67,2002-11-19,0.0,162,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
136180,2016-05-28,571977,N,ES,H,71,2008-10-02,0.0,91,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
136181,2016-05-28,413223,N,ES,H,42,2004-09-24,0.0,172,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
136182,2016-05-28,386998,N,ES,H,46,2002-10-22,0.0,163,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1


In [16]:
test.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1', 'rowNB'],
      dtype='object')

In [17]:
test.fecha_dato.describe()

count         136184
unique             1
top       2016-05-28
freq          136184
Name: fecha_dato, dtype: object

In [18]:
train.fecha_dato.describe()

count         269591
unique            16
top       2016-04-28
freq           19937
Name: fecha_dato, dtype: object

In [20]:
pd.set_option('display.max_columns', 500)

In [21]:
df_train2[df_train2['fecha_dato'] == '2016-04-28']

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB,fecha_dato_next,ncodpers_next,ind_empleado_next,pais_residencia_next,sexo_next,age_next,fecha_alta_next,ind_nuevo_next,antiguedad_next,indrel_next,ult_fec_cli_1t_next,indrel_1mes_next,tiprel_1mes_next,indresi_next,indext_next,conyuemp_next,canal_entrada_next,indfall_next,tipodom_next,cod_prov_next,nomprov_next,ind_actividad_cliente_next,renta_next,segmento_next,ind_ahor_fin_ult1_next,ind_aval_fin_ult1_next,ind_cco_fin_ult1_next,ind_cder_fin_ult1_next,ind_cno_fin_ult1_next,ind_ctju_fin_ult1_next,ind_ctma_fin_ult1_next,ind_ctop_fin_ult1_next,ind_ctpp_fin_ult1_next,ind_deco_fin_ult1_next,ind_deme_fin_ult1_next,ind_dela_fin_ult1_next,ind_ecue_fin_ult1_next,ind_fond_fin_ult1_next,ind_hip_fin_ult1_next,ind_plan_fin_ult1_next,ind_pres_fin_ult1_next,ind_reca_fin_ult1_next,ind_tjcr_fin_ult1_next,ind_valo_fin_ult1_next,ind_viv_fin_ult1_next,ind_nomina_ult1_next,ind_nom_pens_ult1_next,ind_recibo_ult1_next,rowNB_next,rowDif
111,2016-04-28,16099,N,ES,V,48,1995-02-22,0.0,254,1.0,NaN,1.0,A,S,N,NaN,KAT,N,1.0,28.0,MADRID,1.0,215872.71,02 - PARTICULARES,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,0,0.0,0.0,1,1,2015-11-28,16099,N,ES,V,48,1995-02-22,0.0,249,1.0,NaN,1.0,A,S,N,NaN,KAT,N,1.0,28.0,MADRID,1.0,215872.71,02 - PARTICULARES,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,1,0,0,0.0,0.0,1,2,1
498,2016-04-28,16925,N,ES,V,67,1995-05-03,0.0,240,1.0,NaN,1.0,I,S,N,NaN,KAT,N,1.0,28.0,MADRID,0.0,NaN,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1,2015-08-28,16925,N,ES,V,66,1995-05-03,0.0,233,1.0,NaN,1.0,I,S,N,NaN,KAT,N,1.0,28.0,MADRID,0.0,NaN,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,2,1
522,2016-04-28,16952,N,ES,V,69,1995-05-04,0.0,252,1.0,NaN,1.0,I,S,N,NaN,KAT,N,1.0,28.0,MADRID,0.0,427165.44,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1,2015-02-28,16952,N,ES,V,69,1995-05-04,0.0,243,1.0,NaN,1.0,I,S,N,NaN,KAT,N,1.0,28.0,MADRID,0.0,427165.44,02 - PARTICULARES,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,2,1
584,2016-04-28,17074,N,ES,V,59,1997-06-03,0.0,227,1.0,NaN,1.0,A,S,N,NaN,KAT,N,1.0,28.0,MADRID,1.0,75063.27,01 - TOP,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0.0,0.0,0,4,2015-11-28,17074,N,ES,V,58,1997-06-03,0.0,222,1.0,NaN,1.0,A,S,N,NaN,KAT,N,1.0,28.0,MADRID,1.0,75063.27,01 - TOP,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0.0,0.0,0,5,1
710,2016-04-28,17370,N,ES,V,71,1995-07-28,0.0,249,1.0,NaN,1.0,I,S,N,NaN,KAT,N,1.0,45.0,TOLEDO,1.0,91885.68,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,2,2015-07-28,17370,N,ES,V,70,1995-07-28,0.0,240,1.0,NaN,1.0,I,S,N,NaN,KAT,N,1.0,45.0,TOLEDO,1.0,91885.68,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804915,2016-04-28,1540274,N,ES,V,53,2016-03-17,1.0,1,1.0,NaN,1.0,A,S,N,NaN,KHM,N,1.0,38.0,SANTA CRUZ DE TENERIFE,1.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1,2016-03-28,1540274,N,ES,V,53,2016-03-17,1.0,0,1.0,

# aggregate by max fecha dato train

In [39]:
#df_train2['fecha_dato_datetime'] = df_train2['fecha_dato'].

In [34]:
df_train2.sort_values('fecha_dato').groupby('ncodpers').tail(1)[df_train2.ncodpers == 678356]

<ipython-input-34-707a8253b6c6>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train2.sort_values('fecha_dato').groupby('ncodpers').tail(1)[df_train2.ncodpers == 678356]


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB,fecha_dato_1_before,ncodpers_1_before,ind_empleado_1_before,pais_residencia_1_before,sexo_1_before,age_1_before,fecha_alta_1_before,ind_nuevo_1_before,antiguedad_1_before,indrel_1_before,ult_fec_cli_1t_1_before,indrel_1mes_1_before,tiprel_1mes_1_before,indresi_1_before,indext_1_before,conyuemp_1_before,canal_entrada_1_before,indfall_1_before,tipodom_1_before,cod_prov_1_before,nomprov_1_before,ind_actividad_cliente_1_before,renta_1_before,segmento_1_before,ind_ahor_fin_ult1_1_before,ind_aval_fin_ult1_1_before,ind_cco_fin_ult1_1_before,ind_cder_fin_ult1_1_before,ind_cno_fin_ult1_1_before,ind_ctju_fin_ult1_1_before,ind_ctma_fin_ult1_1_before,ind_ctop_fin_ult1_1_before,ind_ctpp_fin_ult1_1_before,ind_deco_fin_ult1_1_before,ind_deme_fin_ult1_1_before,ind_dela_fin_ult1_1_before,ind_ecue_fin_ult1_1_before,ind_fond_fin_ult1_1_before,ind_hip_fin_ult1_1_before,ind_plan_fin_ult1_1_before,ind_pres_fin_ult1_1_before,ind_reca_fin_ult1_1_before,ind_tjcr_fin_ult1_1_before,ind_valo_fin_ult1_1_before,ind_viv_fin_ult1_1_before,ind_nomina_ult1_1_before,ind_nom_pens_ult1_1_before,ind_recibo_ult1_1_before,rowNB_1_before,rowDif
308577,2015-12-28,678356,N,ES,V,52,2007-02-20,0.0,106,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,3,2016-04-28,678356,N,ES,V,52,2007-02-20,0.0,110,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,4,1


In [35]:
df_train2.sort_values('rowNB').groupby('ncodpers').tail(1)[df_train2.ncodpers == 678356]

<ipython-input-35-073d66ba66a2>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train2.sort_values('rowNB').groupby('ncodpers').tail(1)[df_train2.ncodpers == 678356]


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB,fecha_dato_1_before,ncodpers_1_before,ind_empleado_1_before,pais_residencia_1_before,sexo_1_before,age_1_before,fecha_alta_1_before,ind_nuevo_1_before,antiguedad_1_before,indrel_1_before,ult_fec_cli_1t_1_before,indrel_1mes_1_before,tiprel_1mes_1_before,indresi_1_before,indext_1_before,conyuemp_1_before,canal_entrada_1_before,indfall_1_before,tipodom_1_before,cod_prov_1_before,nomprov_1_before,ind_actividad_cliente_1_before,renta_1_before,segmento_1_before,ind_ahor_fin_ult1_1_before,ind_aval_fin_ult1_1_before,ind_cco_fin_ult1_1_before,ind_cder_fin_ult1_1_before,ind_cno_fin_ult1_1_before,ind_ctju_fin_ult1_1_before,ind_ctma_fin_ult1_1_before,ind_ctop_fin_ult1_1_before,ind_ctpp_fin_ult1_1_before,ind_deco_fin_ult1_1_before,ind_deme_fin_ult1_1_before,ind_dela_fin_ult1_1_before,ind_ecue_fin_ult1_1_before,ind_fond_fin_ult1_1_before,ind_hip_fin_ult1_1_before,ind_plan_fin_ult1_1_before,ind_pres_fin_ult1_1_before,ind_reca_fin_ult1_1_before,ind_tjcr_fin_ult1_1_before,ind_valo_fin_ult1_1_before,ind_viv_fin_ult1_1_before,ind_nomina_ult1_1_before,ind_nom_pens_ult1_1_before,ind_recibo_ult1_1_before,rowNB_1_before,rowDif
308577,2015-12-28,678356,N,ES,V,52,2007-02-20,0.0,106,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,3,2016-04-28,678356,N,ES,V,52,2007-02-20,0.0,110,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,4,1


In [36]:
df_train2[df_train2.ncodpers == 678356]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB,fecha_dato_1_before,ncodpers_1_before,ind_empleado_1_before,pais_residencia_1_before,sexo_1_before,age_1_before,fecha_alta_1_before,ind_nuevo_1_before,antiguedad_1_before,indrel_1_before,ult_fec_cli_1t_1_before,indrel_1mes_1_before,tiprel_1mes_1_before,indresi_1_before,indext_1_before,conyuemp_1_before,canal_entrada_1_before,indfall_1_before,tipodom_1_before,cod_prov_1_before,nomprov_1_before,ind_actividad_cliente_1_before,renta_1_before,segmento_1_before,ind_ahor_fin_ult1_1_before,ind_aval_fin_ult1_1_before,ind_cco_fin_ult1_1_before,ind_cder_fin_ult1_1_before,ind_cno_fin_ult1_1_before,ind_ctju_fin_ult1_1_before,ind_ctma_fin_ult1_1_before,ind_ctop_fin_ult1_1_before,ind_ctpp_fin_ult1_1_before,ind_deco_fin_ult1_1_before,ind_deme_fin_ult1_1_before,ind_dela_fin_ult1_1_before,ind_ecue_fin_ult1_1_before,ind_fond_fin_ult1_1_before,ind_hip_fin_ult1_1_before,ind_plan_fin_ult1_1_before,ind_pres_fin_ult1_1_before,ind_reca_fin_ult1_1_before,ind_tjcr_fin_ult1_1_before,ind_valo_fin_ult1_1_before,ind_viv_fin_ult1_1_before,ind_nomina_ult1_1_before,ind_nom_pens_ult1_1_before,ind_recibo_ult1_1_before,rowNB_1_before,rowDif
308567,2015-02-28,678356,N,ES,V,51,2007-02-20,0.0,101,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1,2015-03-28,678356,N,ES,V,51,2007-02-20,0.0,101,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,2,1
308572,2015-03-28,678356,N,ES,V,51,2007-02-20,0.0,101,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,2,2015-12-28,678356,N,ES,V,52,2007-02-20,0.0,106,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,3,1
308577,2015-12-28,678356,N,ES,V,52,2007-02-20,0.0,106,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,3,2016-04-28,678356,N,ES,V,52,2007-02-20,0.0,110,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,4,1


In [38]:
train[train.ncodpers == 678356]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,rowNB
63286,2015-03-28,678356,N,ES,V,51,2007-02-20,0.0,101,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1
77081,2016-04-28,678356,N,ES,V,52,2007-02-20,0.0,110,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,2
162514,2015-12-28,678356,N,ES,V,52,2007-02-20,0.0,106,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,3
238011,2015-02-28,678356,N,ES,V,51,2007-02-20,0.0,101,1.0,NaN,1.0,I,S,N,NaN,KAA,N,1.0,8.0,BARCELONA,0.0,NaN,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,4


In [41]:
train_max = train.sort_values('fecha_dato').groupby('ncodpers').tail(1)

In [42]:
train_max.shape

(116342, 49)

In [43]:
test.shape


(136184, 49)

In [44]:
train.shape


(269591, 49)